In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [ ]:
# Предсказание цен на недвижимость в Калифорнии
# Скачан с https://www.kaggle.com/datasets/camnugent/california-housing-prices/data
dataset = pd.read_csv("housing.csv")
dataset

In [ ]:
# Пустые значения есть только в колонке 'total_bedrooms' - заполним их -1
dataset['total_bedrooms'] = dataset['total_bedrooms'].fillna(-1)

# Выделяем целевую переменную (медианная стоимость дома в ближайшем окружении)
feature_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms','total_bedrooms', 'population', 'households', 'median_income','ocean_proximity']
target_column = 'median_house_value'
data = dataset[feature_columns]
target = dataset[target_column]

In [ ]:
# Обернем в функции код обучения и валидации качества моделей (линейной регрессии и random forest)
# Для оценки качества будем смотреть на MAPE (на сколько процентов в среднем ошиблись) и MAE (на сколько в среднем долларов оишблись)
def train_evaluate_lr(train_data, train_target, val_data, val_target):
    model = LinearRegression()
    scaler = StandardScaler()

    # Нормализуем данные перед обучением
    scaled_train = scaler.fit_transform(train_data)
    scaled_val = scaler.transform(val_data)

    model = model.fit(scaled_train, train_target)
    print("Модель обучена")

    val_pred = model.predict(scaled_val)
    mape = np.round(mean_absolute_percentage_error(val_target, val_pred) * 100, 2) # Переведем в проценты
    mae = int(mean_absolute_error(val_target, val_pred))
    print("MAPE:", mape, "%")
    print("MAE:", mae, "$")

    # Вычислим средний вклад фичи в предсказание - коэффициент в модели от фичи
    coef_dict = {feature: np.round(model.coef_[i], 2) for i, feature in enumerate(train_data.columns)}
    sorted_coefs = sorted(coef_dict.items(), key=lambda x: x[1], reverse=True)
    print("Вклад фичей в предсказание:")
    for feature, coef in sorted_coefs:
        print(feature, coef)


def train_evaluate_rf(train_data, train_target, val_data, val_target):
    model = RandomForestRegressor(max_depth=8)

    model = model.fit(train_data, train_target)
    print("Модель обучена")

    val_pred = model.predict(val_data)
    mape = np.round(mean_absolute_percentage_error(val_target, val_pred) * 100, 2) # Переведем в проценты
    mae = int(mean_absolute_error(val_target, val_pred))
    print("MAPE:", mape, "%")
    print("MAE:", mae, "$")

In [ ]:
# Бейзлайн без доп фичей (обучаем на чем есть)
baseline = data[[feature for feature in feature_columns if feature != "ocean_proximity"]]
train_data, val_data, train_target, val_target = train_test_split(baseline, target, test_size=0.2, random_state=42)
train_evaluate_lr(train_data, train_target, val_data, val_target)

In [ ]:
# Добавим свои фичи на простых функциях и комбинациях
basic_feats = data[[feature for feature in feature_columns if feature != "ocean_proximity"]]
# Вычтем из общего количества комнат количество спален - сколько других комнат
basic_feats["other_rooms_num"] = basic_feats['total_rooms'] - basic_feats["total_bedrooms"]

# Поделим население на количество домов - сколько в среднем людей живет в одном доме
basic_feats["avg_people_per_house"] = basic_feats['population'] / basic_feats["households"]

# Умножим население на медианный доход - сколько в среднем зарабатывают всего в районе
basic_feats["total_income"] = basic_feats['median_income'] * basic_feats["households"]

train_data, val_data, train_target, val_target = train_test_split(basic_feats, target, test_size=0.2, random_state=42)
train_evaluate_lr(train_data, train_target, val_data, val_target)

In [ ]:
# Возьмем 3 наиболее используемые фичи и разобьем их на бины
bin_features = data[[feature for feature in feature_columns if feature not in ["ocean_proximity"]]]

# Одинаковый размер промежутков - uniform, можно quantile для разделения по квантилям
bins = KBinsDiscretizer(n_bins=4, strategy="uniform")
bin_features[bins.get_feature_names_out()]= bins.fit_transform(bin_features[["latitude", "longitude", "median_income"]]).toarray()

train_data, val_data, train_target, val_target = train_test_split(bin_features, target, test_size=0.2, random_state=42)
train_evaluate_lr(train_data, train_target, val_data, val_target)

In [ ]:
# Поработаем с координатами - попробуем добавить расстояние до центров крупных городов Калифорнии
# Недвижимость там явно дороже
cities = {
    "SC": (38.57, -121.49),
    "SF": (37.78, -122.42),
    "SJ": (37.33, -121.88),
    "LA": (34.05, -118.24),
    "SD": (32.71, -117.16),
    "corner": (41.99, -114.04)
}

# Вычисляем евклидово расстояние ("по прямой"). Корректнее вычислять расстояние по дуге, так как Земля - шар
# Haversine - функция, позволяющая географически верно вычислить расстояние
def add_dist_to_cities(df, cities):
    for city in cities.keys():
        df[f"distance_to_{city}"] = np.sqrt((df["latitude"] - cities[city][0])**2 + (df["longitude"] - cities[city][1])**2) 
    
    return df

dist_features = data[[feature for feature in feature_columns if feature not in ["ocean_proximity"]]]
dist_features = add_dist_to_cities(dist_features, cities)

train_data, val_data, train_target, val_target = train_test_split(dist_features, target, test_size=0.2, random_state=42)
train_evaluate_lr(train_data, train_target, val_data, val_target)

In [ ]:
# А теперь посмотрим как добавление фичей влияет на Random Forest
print("Базовый вариант - без доп фичей")
train_data, val_data, train_target, val_target = train_test_split(baseline, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

print("С фичами из простых функций")
train_data, val_data, train_target, val_target = train_test_split(basic_feats, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

print("С фичами-бинами")
train_data, val_data, train_target, val_target = train_test_split(bin_features, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

print("С фичами расстояний")
train_data, val_data, train_target, val_target = train_test_split(dist_features, target, test_size=0.2, random_state=42)
train_evaluate_rf(train_data, train_target, val_data, val_target)

In [ ]:
# Оценим важности фичей при помощи корреляции
from sklearn.feature_selection import r_regression

corr = r_regression(train_data, train_target, center=True)

# Отсортируем и сопоставим с фичами
coef_dict = {feature: np.round(abs(corr[i]), 2) for i, feature in enumerate(train_data.columns)}
sorted_coefs = sorted(coef_dict.items(), key=lambda x: x[1], reverse=True)
print("Вклад фичей в предсказание:")
for feature, coef in sorted_coefs:
    print(feature, coef)

In [ ]:
train_evaluate_lr(
    train_data[[column for column in train_data.columns if column !="distance_to_corner"]],
    train_target,
    val_data[[column for column in train_data.columns if column !="distance_to_corner"]],
    val_target
)

In [ ]:
# У нас есть категориальная фича, которую до этого не использовали
dataset["ocean_proximity"].value_counts()

In [ ]:
# Закодируем двумя методами
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
oh = OneHotEncoder(sparse_output=False)
te = TargetEncoder(target_type="continuous")

oh_train_data, oh_val_data, train_target, val_target = train_test_split(data, target, test_size=0.2, random_state=42)
oh_train_data[oh.get_feature_names_out()] = oh.fit_transform(oh_train_data[["ocean_proximity"]])
oh_val_data[oh.get_feature_names_out()] = oh.transform(oh_val_data[["ocean_proximity"]])
# Убираем оригинальную колонку перед обучением
oh_train_data, oh_val_data = oh_train_data.drop(columns=["ocean_proximity"]), oh_val_data.drop(columns=["ocean_proximity"])

te_train_data, te_val_data, train_target, val_target = train_test_split(data, target, test_size=0.2, random_state=42)
te_train_data[te.get_feature_names_out()] = te.fit_transform(te_train_data[["ocean_proximity"]], train_target)
te_val_data[te.get_feature_names_out()] = te.transform(te_val_data[["ocean_proximity"]])

In [ ]:
# Обучим 2 линейные регрессии:
print("С One-hot экодером:")
train_evaluate_lr(oh_train_data, train_target, oh_val_data, val_target)

print("С Target экодером:")
train_evaluate_lr(te_train_data, train_target, te_val_data, val_target)

In [ ]:
# Обучим 2 Random Forest:
print("С One-hot экодером:")
train_evaluate_rf(oh_train_data, train_target, oh_val_data, val_target)

print("С Target экодером:")
train_evaluate_rf(te_train_data, train_target, te_val_data, val_target)